In [48]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure your code runs with RUN ALL. It should load the data that you're given, do all the processing, and spit out results. Comment out or remove anything that you've cleaned up and don't need - e.g. if you scaled a value manually, then moved that into a pipeline, don't leave the original scaling code active when the file is run.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a markdown cell at the bottom, and put in a few notes addressing the following:
    <ul>
        <li> Accuracy of your models with/without feature selection. Include both train/test for each. Please use R2 and RMSE. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Hyperparameter Changes / Grid Search Improvements. What did you try, and why. Similar explaination to above, short. 
        <li> Overall this section should be roughly as long as this intro block - just outline what the results were, what you did to improve, and the results after. 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

In [49]:
#Load Data
df = pd.read_csv("data/Euro_Salary.csv")
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,24/11/2020 13:55:19,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df.sample(10)

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
388,24/11/2020 19:07:37,35.0,Male,Hamburg,Software Engineer,13,3,Lead,Scala / Python,"Python, Ruby, Java / Scala, Google Cloud, Kube...",95000.0,15000,25,Full-time employee,Unlimited contract,English,51-100,Startup
805,26/11/2020 22:44:06,34.0,Female,Berlin,QA Engineer,1 (as QA Engineer) / 11 in total,11,Senior,Python,"AWS, Kubernetes, Docker",60000.0,2000,28,Full-time employee,Unlimited contract,German,1000+,Product
943,30/11/2020 12:10:34,26.0,Male,Lisbon,Software Engineer,4,0,Middle,JavaScript,"Javascript / Typescript, Ruby, SQL",21000.0,0,22,Full-time employee,Unlimited contract,English,51-100,Product
134,24/11/2020 12:06:58,40.0,Male,Berlin,Frontend Developer,7,1,Middle,Javascript,"PHP, Javascript / Typescript, Dart",54000.0,NaN,24,Full-time employee,Unlimited contract,English,11-50,Product
568,25/11/2020 12:44:24,27.0,Male,Berlin,DevOps,4,4,Middle,AWS,"Python, AWS, Docker",66000.0,6000,30,Full-time employee,Unlimited contract,English,101-1000,Consulting / Agency
418,24/11/2020 20:26:20,NaN,Male,Munich,Software Engineer,4,3,Middle,Java,"Javascript / Typescript, AWS, Kubernetes",56000.0,0,NaN,Full-time employee,Unlimited contract,English,1000+,Product
716,25/11/2020 23:32:29,26.0,Male,London,DevOps,3,0,Middle,"networking, linux, automation, cloud","Python, AWS",37000.0,NaN,25,Full-time employee,Unlimited contract,English,51-100,Product
995,30/11/2020 21:48:26,32.0,Male,Berlin,Data Engineer,7,5,Senior,Python/SQL,"AWS, Kubernetes, Docker",82000.0,82000,30,Full-time employee,Unlimited contract,English,101-1000,Product
329,24/11/2020 17:29:25,33.0,Male,Hamburg,Software Engineer,10,3,Senior,Ruby,Elixir,75000.0,NaN,30,Full-time employee,Unlimited contract,English,101-1000,Product
548,25/11/2020 11:47:34,33.0,Male,Berlin,Software Engineer,16,5,Senior,Java,"Python, Kotlin, Java / Scala, Go, Rust, AWS, K...",83000.0,17000,27,Full-time employee,Unlimited contract,English,1000+,Product


In [63]:
#df.info()
df["Other technologies/programming languages you use often"].isna().value_counts()

False    1096
True      157
Name: Other technologies/programming languages you use often, dtype: int64

In [52]:
df.head()

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
0,24/11/2020 11:14:15,26.0,Male,Munich,Software Engineer,5,3,Senior,TypeScript,"Kotlin, Javascript / Typescript",80000.0,5000,30,Full-time employee,Unlimited contract,English,51-100,Product
1,24/11/2020 11:14:16,26.0,Male,Berlin,Backend Developer,7,4,Senior,Ruby,NaN,80000.0,NaN,28,Full-time employee,Unlimited contract,English,101-1000,Product
2,24/11/2020 11:14:21,29.0,Male,Berlin,Software Engineer,12,6,Lead,Javascript / Typescript,"Javascript / Typescript, Docker",120000.0,120000,30,Self-employed (freelancer),Temporary contract,English,101-1000,Product
3,24/11/2020 11:15:24,28.0,Male,Berlin,Frontend Developer,4,1,Junior,Javascript,NaN,54000.0,NaN,24,Full-time employee,Unlimited contract,English,51-100,Startup
4,24/11/2020 11:15:46,37.0,Male,Berlin,Backend Developer,17,6,Senior,C# .NET,".NET, SQL, AWS, Docker",62000.0,NaN,29,Full-time employee,Unlimited contract,English,101-1000,Product


In [53]:
def is_float(element: any) -> bool:
    #If you expect None to be passed:
    if element is None: 
        return False
    try:
        float(element)
        return True
    except ValueError:
        return False

In [85]:
df.dropna(inplace=True)
for i in range(len(df)):
    if is_float(df['Yearly bonus + stocks in EUR'].iloc[i]) == False:
        df['Yearly bonus + stocks in EUR'].iloc[i] = 0

df['Yearly bonus + stocks in EUR'] = df['Yearly bonus + stocks in EUR'].astype('float64')
df['Yearly bonus + stocks in EUR'].replace(np.nan, 0, inplace=True)
df['Total Compensation'] = df['Yearly brutto salary (without bonus and stocks) in EUR'] + df['Yearly bonus + stocks in EUR']

In [87]:
model = LinearRegression(n_jobs=-1)
imputer_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="mean"))
])

numeric_transformer = Pipeline(steps=[
        ("scaler", MinMaxScaler())
])

preprocessor = Pipeline(steps=[
        ('imp', imputer_transformer),
        ("num", numeric_transformer),
])

pipeline_steps = [('pre', preprocessor),('LR', model)]

best = Pipeline(pipeline_steps)
best.fit(xTrain,yTrain)

NameError: name 'SimpleImputer' is not defined

# Answers and Explainations
(Expand/modify as needed)

### Results

### Feature Selection Activities

### Hyperparameter Changes